In [1]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from dash import html
from dash import dcc
from jupyter_dash import JupyterDash
import dash
from dash.dependencies import Input, Output

In [2]:
companies=pd.read_csv('2021_registered_companies.csv')

In [3]:
companies['state'].value_counts()

Maharashtra             10077
Uttar Pradesh            5772
Delhi                    5449
Karnataka                4213
Telangana                3655
Tamil Nadu               3515
Gujarat                  3057
Haryana                  2705
West Bengal              2551
Kerala                   2084
Bihar                    1997
Rajasthan                1979
Madhya Pradesh           1578
Andhra Pradesh           1184
Odisha                   1110
Punjab                    771
Jharkhand                 647
Assam                     440
Chhattisgarh              437
Uttarakhand               432
Jammu & Kashmir           331
Himachal Pradesh          244
Chandigarh                220
Goa                       153
Manipur                    83
Pondicherry                49
Tripura                    39
Arunachal Pradesh          23
Nagaland                   21
Dadra & Nagar Haveli       18
Meghalaya                  18
Andaman & Nicobar          15
Daman and Diu              11
Mizoram   

In [4]:
foreign=companies[companies['company_type']=='Subsidiary of Foreign Company']

In [5]:
month_date=pd.DataFrame(companies['date_of_registration'].value_counts())
month_date.reset_index(inplace=True)
month_date['month']='month'
month_date.sort_values('index',inplace=True)

month_date_fr=pd.DataFrame(foreign['date_of_registration'].value_counts())
month_date_fr.reset_index(inplace=True)
month_date_fr['month']='month'
month_date_fr.sort_values('index',inplace=True)

def assign_month(df):
    for i in range(0,len(df.index)):
        if '/01' in df.loc[i,'index']:
            df.loc[i,'month']='Jan-21'
        elif '/02' in df.loc[i,'index']:
            df.loc[i,'month']='Feb-21'
        elif '/03' in df.loc[i,'index']:
            df.loc[i,'month']='Mar-21'
        elif '/04' in df.loc[i,'index']:
            df.loc[i,'month']='Apr-21'
            
assign_month(month_date)
assign_month(month_date_fr)

In [6]:
companies_activity=companies.pivot_table(index='activity_description',values=['paidup_capital','authorized_capital'])
companies_activity.reset_index(inplace=True)
capitals = companies_activity.melt('activity_description', var_name='capital',  value_name='vals')

companies_activity_fr=foreign.pivot_table(index='activity_description',values=['paidup_capital','authorized_capital'])
companies_activity_fr.reset_index(inplace=True)
capitals_fr = companies_activity_fr.melt('activity_description', var_name='capital',  value_name='vals')

In [7]:
sectors=pd.DataFrame(companies['activity_description'].value_counts())
sectors.reset_index(inplace=True)
sectors_fr=pd.DataFrame(foreign['activity_description'].value_counts())
sectors_fr.reset_index(inplace=True)

In [8]:
state_data=pd.DataFrame(companies['state'].value_counts())
state_data.reset_index(inplace=True)
state_data.rename(columns = {'index':'State','state':'Companies_Registered'}, inplace = True)
fig = px.choropleth(
            state_data,
            geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
            featureidkey='properties.ST_NM',
            locations='State',
            color='Companies_Registered',
            color_continuous_scale='Reds', height=700
            )

fig=fig.update_geos(fitbounds="locations", visible=True)

In [9]:
state_data.head()

,State,Companies_Registered
0,Maharashtra,10077
1,Uttar Pradesh,5772
2,Delhi,5449
3,Karnataka,4213
4,Telangana,3655


In [10]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)     #initializing the app
server=app.server                                                          #creating server variable 

app.layout = html.Div([
    
    html.H1([
        "Indian Startups-2021"
    ],style={'background': '#6392e7','padding':'20px 20px 0px 20px','margin':'0','text-align':'center'}),
    
    dcc.Tabs(id='tabs', value='bar', children=[
        dcc.Tab(label='Bar Charts', value='bar',style={'background':'#6392e7','border-style':'none'}),
        dcc.Tab(label='Geographical Plot', value='geo',style={'background':'#6392e7','border-style':'none'}),
        dcc.Tab(label='Other Charts', value='other',style={'background':'#6392e7','border-style':'none'}),
        
    ],style={'font-size':'20px'}),
    html.Div(id='plots')
        
],style={'backgroundColor': '#f5f4f0','margin':'0' })


@app.callback(
    Output('plots', 'children'),
    Input('tabs', 'value')
)
def render_content(tab):
    if tab == 'bar':
        return  html.Div([       #row
        html.Div([               #col 1
            html.Div([           #box 1
                dcc.Tabs(id='bar', value='month_name',vertical=True, children=[
                dcc.Tab(label='Month', value='month_name',
                        style={'backgroundColor':'#B0B0B0','border-style':'none'}),
                dcc.Tab(label='States', value='state',
                        style={'backgroundColor':'#B0B0B0','border-style':'none'}),
                dcc.Tab(label='Registrar of Companies (ROC)', value='roc',
                        style={'backgroundColor':'#B0B0B0','border-style':'none'}),
                dcc.Tab(label='Category', value='category',
                        style={'backgroundColor':'#B0B0B0','border-style':'none'}),
                dcc.Tab(label='Class', value='class',
                        style={'backgroundColor':'#B0B0B0','border-style':'none'}),
                dcc.Tab(label='Company Type', value='company_type',
                        style={'backgroundColor':'#B0B0B0','border-style':'none'}),
                dcc.Tab(label='Activity', value='activity_description',
                        style={'backgroundColor':'#B0B0B0','border-style':'none','margin-bottom':'15px'})
                ]),

                dcc.RadioItems(
                            id='type_toggle',
                            options=[{'label': 'All', 'value': 'all'},
                                     {'label': 'Foreign', 'value': 'foreign'}
                                    ],
                            value='all',
                            labelStyle={'display': 'inline-block'}
                )
            ],style={'text-align': 'center','font-size':'16px','padding':'15px','margin':'0'}) #box 1 close
        ],style={'width':'20%','backgroundColor':'#B0B0B0'}),                                  #col 1 close
         
        html.Div([                                 #col 2
            html.Div([                             #box 2
                dcc.Graph(id='Graph3'),
               
                dcc.Slider(
                    id='no_of_states_slider',
                    min=1,
                    value=5,
                    step=4)
            ],style={ 'padding':'15px'})              #box 2 close
        ],style={'width':'80%'}),                     #col 2 close
    ],style={'display': 'flex','row-gap': '30px'})    #row close
    
    
    elif tab == 'geo':
        return html.Div([
            dcc.Graph(id='geo',figure=fig)
        ])
    
    
    elif tab == 'other':
        return html.Div([        #row
            
            html.Div([           #col 1
                html.Div([       #box 1
                    
                    dcc.Tabs(id='other_graphs', value='line',vertical=True, children=[
                        dcc.Tab(label='Line Chart', value='line',
                            style={'backgroundColor':'#B0B0B0','border-style':'none'}),
                        dcc.Tab(label='Pie Chart', value='pie',
                            style={'backgroundColor':'#B0B0B0','border-style':'none'}),
                    
                    ]),
                             
                    dcc.RadioItems(
                        id='type',
                        options=[{'label': 'All', 'value': 'all'},
                                    {'label': 'Foreign', 'value': 'foreign'}
                                ],
                        value='all',
                        labelStyle={'display': 'inline-block'}
                    )
                        
                ],style={'text-align': 'center','font-size':'16px','padding':'15px','margin':'0'}) #box 1 close
            ],style={'width':'20%','backgroundColor':'#B0B0B0'}),                                  #col 1 close
         
            html.Div([                                 #col 2
                html.Div([                             #box 2
                    dcc.Graph(id='other_graph'),
                ],style={ 'padding':'15px'})              #box 2 close
            ],style={'width':'80%'}),                     #col 2 close
            
        ],style={'display': 'flex','row-gap': '30px'})    #row close
    
    
           



@app.callback(
    Output('Graph3','figure'),
    Output('no_of_states_slider','max'),
    Output('no_of_states_slider','marks'),
    Input('no_of_states_slider','value'),
    Input('bar', 'value'),
    Input('type_toggle','value')
)
def plots_bar(num,bar,toggle_type):
    df=pd.DataFrame(companies[bar].value_counts().head(num))
    df_foreign=pd.DataFrame(foreign[bar].value_counts().head(num))
    
    if bar=='month_name':
        if toggle_type=='all':
         
            index_feb=list(month_date[month_date['month']=='Feb-21'].index)
            index_mar=list(month_date[month_date['month']=='Mar-21'].index)
            index_apr=list(month_date[month_date['month']=='Apr-21'].index)
            
            if num==1: month_date.drop(index_feb+index_mar+index_apr,inplace=True)
            if num==2: month_date.drop(index_mar+index_apr,inplace=True)
            if num==3: month_date.drop(index_apr,inplace=True)
            
            fig = make_subplots(rows=1, cols=2)
            fig.add_trace(
                go.Bar(x=df.reindex(['Jan-21','Feb-21','Mar-21','Apr-21']).reset_index()['index'],
                       y=df.reindex(['Jan-21','Feb-21','Mar-21','Apr-21']).reset_index()['month_name']),
                row=1, col=1
            )

            fig.add_trace(
                go.Box(x=month_date['month'],y=month_date['date_of_registration']),
                row=1, col=2
            )
            fig.update_xaxes(title_text="Month", row=1, col=1)
            fig.update_xaxes(title_text="Month", row=1, col=2)
            
            fig.update_yaxes(title_text="Number of Companies Registered", row=1, col=1)
            fig.update_yaxes(title_text="Number of Companies Registered in a day", row=1, col=2)
            fig.update_layout(height=700, showlegend=False)
            
        if toggle_type=='foreign':
           
            index_feb=list(month_date_fr[month_date_fr['month']=='Feb-21'].index)
            index_mar=list(month_date_fr[month_date_fr['month']=='Mar-21'].index)
            index_apr=list(month_date_fr[month_date_fr['month']=='Apr-21'].index)
            
            if num==1: month_date_fr.drop(index_feb+index_mar+index_apr,inplace=True)
            if num==2: month_date_fr.drop(index_mar+index_apr,inplace=True)
            if num==3: month_date_fr.drop(index_apr,inplace=True)
            
            fig = make_subplots(rows=1, cols=2)
            fig.add_trace(
                go.Bar(x=df_foreign.reindex(['Jan-21','Feb-21','Mar-21','Apr-21']).reset_index()['index'],
                       y=df_foreign.reindex(['Jan-21','Feb-21','Mar-21','Apr-21']).reset_index()['month_name']),
                row=1, col=1
            )

            fig.add_trace(
                go.Box(x=month_date_fr['month'],y=month_date_fr['date_of_registration']),
                row=1, col=2
            )
            fig.update_xaxes(title_text="Month", row=1, col=1)
            fig.update_xaxes(title_text="Month", row=1, col=2)
            
            fig.update_yaxes(title_text="Number of Companies Registered", row=1, col=1)
            fig.update_yaxes(title_text="Number of Companies Registered in a day", row=1, col=2)
            
            fig.update_layout(height=700, showlegend=False)
            
    
    else:
        if toggle_type=='all':
            fig=px.bar(df,y=bar,height=700,
                       labels={'index':bar,bar:'Number of Companies Registered'})

        if toggle_type=='foreign':
            fig=px.bar(df_foreign,y=bar,height=700,
                       labels={'index':bar,bar:'Number of Companies Registered'})
            
    maximum=companies[bar].nunique()
    marks={str(num): str(num) for num in range(1,maximum+1)}
    
    return fig,maximum,marks


@app.callback(
    Output('other_graph','figure'),
    Input('other_graphs', 'value'),
    Input('type','value')
)
def plot_others(other_graphs,type):
    if other_graphs == 'line':
        if type=='all':
            fig=px.line(capitals,x="activity_description", y="vals",color='capital',height=800)
        if type=='foreign':
            fig=px.line(capitals_fr,x="activity_description", y="vals",color='capital',height=800)
    if other_graphs == 'pie':
        if type=='all':
            fig=px.pie(sectors,values='activity_description',names='index')
        if type=='foreign':
            fig=px.pie(sectors_fr,values='activity_description',names='index')
    
    return fig




In [11]:
app.run_server(port=8080,debug=True)

Dash app running on http://127.0.0.1:8080/
